In [1]:
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
import torch
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

/home/mahdi/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-10 14:46:53.204114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739186213.220629  114778 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739186213.225057  114778 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-10 14:46:53.243180: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations

In [2]:
device = torch.device('cpu')

In [3]:
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=4).to(device)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
dataset = load_dataset('imagefolder',data_dir=r'/home/mahdi/Desktop/Project/Brain-Tumur-MRI-Detection/dataset')

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4569
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1143
    })
})

In [8]:
def preprocessing(examples):
    images = [image.convert('RGB') for image in examples['image']]
    inputs = processor(images=images, return_tensors='pt').to(device)
    inputs['label'] = examples['label']
    return inputs

processed_train = dataset['train'].map(preprocessing, batched=True)
processed_valid = dataset['test'].map(preprocessing, batched=True)

Map: 100%|██████████| 1143/1143 [00:07<00:00, 145.73 examples/s]


In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    num_train_epochs=5
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_train,
)

In [10]:
trainer.train()

Step,Training Loss
500,0.197900
1000,0.023000


TrainOutput(global_step=1430, training_loss=0.07844353038947899, metrics={'train_runtime': 3152.8449, 'train_samples_per_second': 7.246, 'train_steps_per_second': 0.454, 'total_flos': 1.7703362361039667e+18, 'train_loss': 0.07844353038947899, 'epoch': 5.0})

In [13]:
prediction = trainer.predict(processed_valid)
preds = prediction.predictions.argmax(-1)
labels = prediction.label_ids

In [14]:
accuracy = accuracy_score(labels, preds) * 100
precision = precision_score(labels, preds, average='weighted') * 100
recall = recall_score(labels, preds, average='weighted') * 100
f1 = f1_score(labels, preds, average='weighted') * 100

print(f"Accuracy: {round(accuracy, 2)}")
print(f"Precision: {round(precision, 2)}")
print(f"Recall: {round(recall, 2)}")
print(f"F1-Score: {round(f1, 2)}")

Accuracy: 98.6
Precision: 98.6
Recall: 98.6
F1-Score: 98.6


In [15]:
print(classification_report(labels,preds))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       281
           1       0.99      0.97      0.98       243
           2       1.00      0.99      1.00       334
           3       0.98      0.99      0.98       285

    accuracy                           0.99      1143
   macro avg       0.99      0.99      0.99      1143
weighted avg       0.99      0.99      0.99      1143

